In [1]:
import sqlalchemy as sa
import pandas as pd
import pyarrow as pa
import credentials_db as creds

In [2]:
pd.options.mode.copy_on_write = True

In [101]:
string_pa = pd.ArrowDtype(pa.string())

In [3]:
engine_sda = sa.create_engine('oracle+cx_oracle://'+creds.sda['usr']+':'+creds.sda['pwd']+creds.sda['host']+'/'+creds.sda['db'])

In [4]:
txt_for_sql = open('./sql/enrollment data fall census.sql','r').read()

In [5]:
queries = []
for i in range (19,25):
    queries.append(txt_for_sql.replace('__yr__',str(i)))
sql_enrolllment_data_fall = '\nunion\n'.join(queries)

In [63]:
enrollment = pd.read_sql(sql_enrolllment_data_fall,engine_sda,dtype={
        'sytdean_pidm':'uint32[pyarrow]',
        'sytdean_term_code_key':'category',
        'sytdean_registered_ind':'category',
        'sytdean_camp_code':'category',
        'sytdean_levl_code':'category',
        'sytdean_coll_code_1':'category',
        'sytdean_majr_code_1':'category',
        'sytdean_styp_code':'category',
        'sytdean_class_code':'category'
    })

In [135]:
def retention(df):
    return (df
        .assign(
            classification = df.sytdean_styp_code.case_when(caselist = [
                (df.sytdean_styp_code.isin(['F','T']) & (df.sytdean_majr_code_1 == 'PHA6'), df.sytdean_styp_code.astype(string_pa) + '_PHA6'),
                (df.sytdean_styp_code.isin(['F','T']) & (df.sytdean_majr_code_1 == 'NURS'), df.sytdean_styp_code.astype(string_pa) + '_BSN'),
                (df.sytdean_styp_code.isin(['F','T']), df.sytdean_styp_code),
                (df.sytdean_styp_code.isin(['C','I','R']) & (df.sytdean_majr_code_1 == 'PHA6'), df.sytdean_class_code.astype(string_pa) + '_PHA6'),
                (df.sytdean_styp_code.isin(['C','I','R']) & (df.sytdean_majr_code_1 == 'NURS'), df.sytdean_class_code.astype(string_pa) + '_BSN'),
                (df.sytdean_styp_code.isin(['C','I','R']), df.sytdean_class_code.astype(string_pa)),
                (df.sytdean_styp_code.isin(['N','S','V']), 'NSV')
            ]).astype('category'),
            future_term = (df.sytdean_term_code_key.astype('int64[pyarrow]')+100).astype(string_pa).astype('category')
        )
        .merge(df,how='left',left_on=['sytdean_pidm','future_term'],right_on=['sytdean_pidm','sytdean_term_code_key'])
        .drop(['future_term'],axis=1)
    )